In [3]:
import pandas as pd
import numpy as np

BASE_PATH_TO_FOLDER = input("Please provide the path to the root folder of this project: ")

CSV_KANGLE = BASE_PATH_TO_FOLDER + "/data/raw/Portland Crime Data Raw.csv"
CSV_PPB = BASE_PATH_TO_FOLDER + "/data/raw/CrimeData-2023.csv"

In [ ]:
df_kaggle = pd.read_csv(CSV_KANGLE, sep='\t')
df_ppb = pd.read_csv(CSV_PPB)

In [5]:
# Confirmed uniqueness of initial column, renaming to ID
df_kaggle = df_kaggle.rename(columns = {'Unnamed: 0': 'ID'})
df_kaggle['ID'].is_unique

True

In [5]:
# Setting up removal of duplicates from ppb_df
comparison_columns = ['Address', 'CaseNumber', 'OccurDate', 'OccurTime', 'OpenDataLat', 'OpenDataLon', 'OpenDataX', 'OpenDataY', 'ReportDate']
# If the columns listed above all match for a single row, then that row is likely a duplicate and can be ignored
# Creating a column which is a string aggregated by all of the specified column names for a row, will use this to determine if a row is a duplicate or not
df_kaggle['dup_key'] = df_kaggle[comparison_columns].astype(str).agg('|'.join, axis=1)  # using axis=1 to signify row aggregation
df_ppb['dup_key'] = df_ppb[comparison_columns].astype(str).agg('|'.join, axis=1)
# Example row: '700 BLOCK OF SW KING AVE|24-933461|2024-12-31|835|45.522619|-122.695457|7639475.0|684279.0|2024-12-31'

In [6]:
# Creating a dataset where we ignore all the 'dup_key' values which are common between both df_ppb & df_kaggle
df_ppb_cleaned = df_ppb[~df_ppb['dup_key'].isin(df_kaggle['dup_key'])]
# Found that the datasets showed absolutely no duplicates, both dataframes still show 63425 rows

In [8]:
# Examining a few rows trying to determine why there were not matches based on the dup_key strings
df_ppb.iloc[0, -1]
df_kaggle.iloc[0, -1]
df_kaggle['dup_key'].isin(['nan|23-X5791168|1/1/2023|246|nan|nan|nan|nan|1/1/2023']).any()

False

In [11]:
# Went to Excel to find exact matching rows based on CaseNumbers
# In excel, found the OccurTime has an apostraphe in front of it, but this doesn't translate to the dataframes
# In excel, found no other inconsistencies
# Checking data frames, I now realize the dates are inconsistnet. ISO standard is YYYY-MM-DD, will convert df_ppb
df_kaggle[df_kaggle['CaseNumber'] == '23-896'].iloc[0,-1]
df_ppb[df_ppb['CaseNumber'] == '23-896'].iloc[1,-1]
df_kaggle[df_kaggle['CaseNumber'] == '23-896'].iloc[0,-1]
df_ppb[df_ppb['CaseNumber'] == '23-896'].iloc[1,-1]
df_ppb[df_ppb['CaseNumber'] == '23-X5791168'].iloc[0,-1]  # Checking for apostraphe in OccurTime

'nan|23-X5791168|1/1/2023|246|nan|nan|nan|nan|1/1/2023'

In [13]:
del df_ppb_cleaned

In [15]:
# Cleaning up column data
for col in ['OccurDate', 'ReportDate']:
    df_kaggle[col] = pd.to_datetime(df_kaggle[col], errors='coerce').dt.strftime('%Y-%m-%d')  # coerce here returns NaT instead of failing
    df_ppb[col] = pd.to_datetime(df_ppb[col], errors='coerce').dt.strftime('%Y-%m-%d')

In [17]:
# resetting the dup_key values
df_kaggle['dup_key'] = df_kaggle[comparison_columns].astype(str).agg('|'.join, axis=1)
df_ppb['dup_key'] = df_ppb[comparison_columns].astype(str).agg('|'.join, axis=1)

In [21]:
df_ppb_cleaned = df_ppb[~df_ppb['dup_key'].isin(df_kaggle['dup_key'])]
# now df_ppb shows 63425 rows and df_ppb_cleaned shows 42634 rows
# This is more than I expected considering the missing months are only Aug-Dec
# Checking df_pbb_cleaned, there are still plenty of rows which have data in months where there should not be

In [25]:
# Examining more data, looking at a case which is in df_ppb_cleaned but has an early date
# Checking kaggle and ppb .csv files on 2023-01-01, both have the same amount of entires, 172 so these should all match
# I found that the CaseNumbers are not the same between my new ppb data and the kaggle data. 
# My best guess here is that the CaseNumbers can change and/or update over time. 
# Example: Kaggle data - CaseNumber: 23-448 == PPB Data - CaseNumber: 23-X5791249
# # Both occurred on the same day at the same time with the same offenses in the same neighborhood, assuming these are the same
# Therefore, I cannot use CaseNumbers to help identify duplicates. Case Numbers are also irrelevant for what I need, so its fine
# if they mismatch
df_kaggle[df_kaggle['CaseNumber'] == '23-X5791249']
df_kaggle[df_kaggle['CaseNumber'] == '23-448']
df_ppb[df_ppb['CaseNumber'] == '23-X5791249']
df_ppb_cleaned[df_ppb_cleaned['CaseNumber'] == '23-X5791249']

,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount,dup_key
0,NaN,23-X5791168,Person,Arbor Lodge,2023-01-01,246,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|23-X5791168|2023-01-01|246|nan|nan|nan|nan...
1,NaN,23-X5791169,Person,Old Town/Chinatown,2023-01-01,209,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|23-X5791169|2023-01-01|209|nan|nan|nan|nan...
2,NaN,23-X5791249,Person,Montavilla,2023-01-01,1133,Assault Offenses,Aggravated Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|23-X5791249|2023-01-01|1133|nan|nan|nan|na...
3,NaN,23-X5791249,Society,Montavilla,2023-01-01,1133,Weapon Law Violations,Weapons Law Violations,NaN,NaN,NaN,NaN,2023-01-01,1,nan|23-X5791249|2023-01-01|1133|nan|nan|nan|na...
4,NaN,23-X5791250,Person,Creston-Kenilworth,2023-01-01,649,Assault Offenses,Aggravated Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|23-X5791250|2023-01-01|649|nan|nan|nan|nan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63420,WB SUNSET HWY AT / SYLVAN EXIT,23-225440,Society,Sylvan-Highlands,2023-08-27,1420,Weapon Law Violations,Weapons Law Violations,45.508945,-122.731195,7630181.0,679546.0,2023-08-27,1,WB SUNSET HWY AT / SYLVAN EXIT|23-225440|2023-...
63421,WB SUNSET HWY AT / ZOO EXIT,23-51637,Property,Arlington Heights,2023-02-23,330,Motor Vehicle Theft,Motor Vehicle Theft,45.506744,-122.713355,7634731.0,678618.0,2023-02-25,1,WB SUNSET HWY AT / ZOO EXIT|23-51637|2023-02-2...
63422,WB SUNSET HWY EO / CANYON TUNNEL,23-227984,Person,Goose Hollow,2023-08-30,920,Assault Offenses,Aggravated Assault,45.515555,-122.693709,7639853.0,681692.0,2023-08-30,1,WB SUNSET HWY EO / CANYON TUNNEL|23-227984|202...
63423,WILDWOOD TRAIL AT / KEIL TRAIL,23-40689,Property,Forest Park,2023-02-13,1130,Motor Vehicle Theft,Motor Vehicle Theft,45.540437,-122.736728,7629081.0,691063.0,2023-02-13,1,WILDWOOD TRAIL AT / KEIL TRAIL|23-40689|2023-0...


In [57]:
del df_ppb_cleaned
del comparison_columns

In [59]:
# Running through the dup_keys one more time, without the CseNumber key
comparison_columns = ['Address', 'OccurDate', 'OccurTime', 'OpenDataLat', 'OpenDataLon', 'OpenDataX', 'OpenDataY', 'ReportDate']
df_kaggle['dup_key'] = df_kaggle[comparison_columns].astype(str).agg('|'.join, axis=1)
df_ppb['dup_key'] = df_ppb[comparison_columns].astype(str).agg('|'.join, axis=1)
df_ppb_cleaned = df_ppb[~df_ppb['dup_key'].isin(df_kaggle['dup_key'])]

In [60]:
# Realized the EVEN MORE data is missing. For some reason, the Kaggle data sometimes has Lat/Lon/DataX/DataY/Address locations
# when this data I've collected more recently from PPB does not. I can still add this data by using OccurDate, OccurTime, and ReportDate
# but this could result in some errors as well. The best solution, which I probably should've just started with
# is to create a subset of data from df_pbb which only targets the OccurDate or the ReportDate timeframe between August & Decemeber
# That said, I think I will still be fine just using OccurDate, OccurTime and ReportDate because that should still result in getting the data
# I am missing. I am also concerned at why there is this discrepency when the data that I collected was more recent than the data from the Kraggle
# collection time. 
# Overall, there are ~5000 cases where there is no data for address in the PPB data. I don't think this will result in significant error here
# I am keeping the kaggle data which has more information and I will only potentially lose SOME data between the timeframe ofmissing data

,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount,dup_key
0,NaN,23-X5791168,Person,Arbor Lodge,2023-01-01,246,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|2023-01-01|246|nan|nan|nan|nan|2023-01-01
1,NaN,23-X5791169,Person,Old Town/Chinatown,2023-01-01,209,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|2023-01-01|209|nan|nan|nan|nan|2023-01-01
2,NaN,23-X5791249,Person,Montavilla,2023-01-01,1133,Assault Offenses,Aggravated Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|2023-01-01|1133|nan|nan|nan|nan|2023-01-01
3,NaN,23-X5791249,Society,Montavilla,2023-01-01,1133,Weapon Law Violations,Weapons Law Violations,NaN,NaN,NaN,NaN,2023-01-01,1,nan|2023-01-01|1133|nan|nan|nan|nan|2023-01-01
4,NaN,23-X5791250,Person,Creston-Kenilworth,2023-01-01,649,Assault Offenses,Aggravated Assault,NaN,NaN,NaN,NaN,2023-01-01,1,nan|2023-01-01|649|nan|nan|nan|nan|2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63420,WB SUNSET HWY AT / SYLVAN EXIT,23-225440,Society,Sylvan-Highlands,2023-08-27,1420,Weapon Law Violations,Weapons Law Violations,45.508945,-122.731195,7630181.0,679546.0,2023-08-27,1,WB SUNSET HWY AT / SYLVAN EXIT|2023-08-27|1420...
63421,WB SUNSET HWY AT / ZOO EXIT,23-51637,Property,Arlington Heights,2023-02-23,330,Motor Vehicle Theft,Motor Vehicle Theft,45.506744,-122.713355,7634731.0,678618.0,2023-02-25,1,WB SUNSET HWY AT / ZOO EXIT|2023-02-23|330|45....
63422,WB SUNSET HWY EO / CANYON TUNNEL,23-227984,Person,Goose Hollow,2023-08-30,920,Assault Offenses,Aggravated Assault,45.515555,-122.693709,7639853.0,681692.0,2023-08-30,1,WB SUNSET HWY EO / CANYON TUNNEL|2023-08-30|92...
63423,WILDWOOD TRAIL AT / KEIL TRAIL,23-40689,Property,Forest Park,2023-02-13,1130,Motor Vehicle Theft,Motor Vehicle Theft,45.540437,-122.736728,7629081.0,691063.0,2023-02-13,1,WILDWOOD TRAIL AT / KEIL TRAIL|2023-02-13|1130...


In [65]:
del df_ppb_cleaned
del comparison_columns

In [67]:
# Running through the dup_keys one more time, without the CseNumber key
comparison_columns = ['OccurDate', 'OccurTime', 'ReportDate']
df_kaggle['dup_key'] = df_kaggle[comparison_columns].astype(str).agg('|'.join, axis=1)
df_ppb['dup_key'] = df_ppb[comparison_columns].astype(str).agg('|'.join, axis=1)
df_ppb_cleaned = df_ppb[~df_ppb['dup_key'].isin(df_kaggle['dup_key'])]

In [7]:
# This only made a difference of 42634 --> 42328, approximately 300 rows. 
# I will have to investigate this further but I'm done for now
# Perhaps the I will proceed with the rangedate after I check these .csv files to determine whether OccurDate or 
# ReportDate will be better to work from. 
# A few days later: Range seems like the optimal route to go down at this point

,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount
0,NaN,23-X5791168,Person,Arbor Lodge,1/1/2023,246,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,1/1/2023,1
1,NaN,23-X5791169,Person,Old Town/Chinatown,1/1/2023,209,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,1/1/2023,1
2,NaN,23-X5791249,Person,Montavilla,1/1/2023,1133,Assault Offenses,Aggravated Assault,NaN,NaN,NaN,NaN,1/1/2023,1
3,NaN,23-X5791249,Society,Montavilla,1/1/2023,1133,Weapon Law Violations,Weapons Law Violations,NaN,NaN,NaN,NaN,1/1/2023,1
4,NaN,23-X5791250,Person,Creston-Kenilworth,1/1/2023,649,Assault Offenses,Aggravated Assault,NaN,NaN,NaN,NaN,1/1/2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63420,WB SUNSET HWY AT / SYLVAN EXIT,23-225440,Society,Sylvan-Highlands,8/27/2023,1420,Weapon Law Violations,Weapons Law Violations,45.508945,-122.731195,7630181.0,679546.0,8/27/2023,1
63421,WB SUNSET HWY AT / ZOO EXIT,23-51637,Property,Arlington Heights,2/23/2023,330,Motor Vehicle Theft,Motor Vehicle Theft,45.506744,-122.713355,7634731.0,678618.0,2/25/2023,1
63422,WB SUNSET HWY EO / CANYON TUNNEL,23-227984,Person,Goose Hollow,8/30/2023,920,Assault Offenses,Aggravated Assault,45.515555,-122.693709,7639853.0,681692.0,8/30/2023,1
63423,WILDWOOD TRAIL AT / KEIL TRAIL,23-40689,Property,Forest Park,2/13/2023,1130,Motor Vehicle Theft,Motor Vehicle Theft,45.540437,-122.736728,7629081.0,691063.0,2/13/2023,1


In [13]:
# Report Dates ranging from 08/01/23-12/31/23 from df_ppb will be added into df_kraggle
# Converting report date into a datetime format
for col in ['OccurDate', 'ReportDate']:
    df_ppb[col] = pd.to_datetime(df_ppb[col], errors='coerce').dt.strftime('%Y-%m-%d')

In [21]:
# Reasonable amount of data, reduced from 63425 rows to 25975
# Ratio of rows is 1: 0.41, ratio of days is 1: 0.33, reasonably close together consider data in ppb rows may be weighted
df_ppb_subset_missing = df_ppb[(df_ppb['ReportDate'] >= '2023-08-01') & (df_ppb['ReportDate'] <= '2023-12-31')]

In [35]:
# Merging the data from the subset onto df_kaggle
starting_id = df_kaggle['ID'].max() + 1
df_ppb_subset_missing = df_ppb_subset_missing.copy()  # Original variable is a view, making a hard copy here as a precaution
df_ppb_subset_missing['ID'] = range(starting_id, starting_id + len(df_ppb_subset_missing))  # Creating incrementing ID based on df_kaggle
df_ppb_subset_missing

,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount,ID
2996,NaN,23-X5861341,Property,Hosford-Abernethy,2023-07-31,1000,Vandalism,Vandalism,NaN,NaN,NaN,NaN,2023-08-01,1,561874
2997,NaN,23-X5861344,Person,Brooklyn,2023-08-01,45,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-08-01,1,561875
2998,NaN,23-X5861348,Person,Parkrose,2023-08-01,119,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-08-01,1,561876
3000,NaN,23-X5861394,Person,Hazelwood,2023-08-01,912,Assault Offenses,Simple Assault,NaN,NaN,NaN,NaN,2023-08-01,1,561877
3006,NaN,23-X5861513,Property,Woodstock,2023-07-31,0,Motor Vehicle Theft,Motor Vehicle Theft,NaN,NaN,NaN,NaN,2023-08-01,1,561878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63411,WB I84 FWY AT / NE 28TH AVE,23-288735,Society,Kerns,2023-11-05,203,Weapon Law Violations,Weapons Law Violations,45.530409,-122.637071,7654509.0,686717.0,2023-11-05,1,587844
63412,WB I84 FWY AT / NE 74TH AVE,23-269273,Person,Madison South,2023-10-14,53,Assault Offenses,Simple Assault,45.535703,-122.587109,7667358.0,688312.0,2023-10-14,1,587845
63415,WB I84 FWY EO / NE 53RD AVE,23-237188,Property,Rose City Park,2023-09-08,2035,Vandalism,Vandalism,45.527393,-122.608116,7661898.0,685423.0,2023-09-08,1,587846
63420,WB SUNSET HWY AT / SYLVAN EXIT,23-225440,Society,Sylvan-Highlands,2023-08-27,1420,Weapon Law Violations,Weapons Law Violations,45.508945,-122.731195,7630181.0,679546.0,2023-08-27,1,587847


In [37]:
# Confirmed that the column names are the same before merging
df_merged = pd.concat([df_kaggle, df_ppb_subset_missing], ignore_index=True)  # Merging dfs, ignoring index

In [39]:
df_merged['ID'].is_unique  # Confirming that the ID field is unique after merge

True

In [43]:
# Setting dates for all date related values to check that data seems to have appropriately combined
for col in ['OccurDate', 'ReportDate']:
    df_merged[col] = pd.to_datetime(df_merged[col], errors='coerce').dt.strftime('%Y-%m-%d')

In [47]:
# Checking report subset for various times
df_merged_subset_a = df_merged[(df_merged['ReportDate'] >= '2023-07-30') & (df_merged['ReportDate'] <= '2023-08-01')]
df_merged_subset_b = df_merged[(df_merged['ReportDate'] >= '2023-12-31') & (df_merged['ReportDate'] <= '2024-01-01')]
df_merged_subset_b

,ID,Address,CaseNumber,CrimeAgainst,Neighborhood,OccurDate,OccurTime,OffenseCategory,OffenseType,OpenDataLat,OpenDataLon,OpenDataX,OpenDataY,ReportDate,OffenseCount
56669,139391,1800 BLOCK OF S RIVER DR,24-900057,Property,Downtown,2024-01-01,1821,Larceny Offenses,Theft From Motor Vehicle,45.508830,-122.673349,7645004.0,679100.0,2024-01-01,1
56670,133457,1100 BLOCK OF N HAYDEN MEADOWS DR,24-900005,Property,East Columbia,2024-01-01,1500,Fraud Offenses,Credit Card/ATM Fraud,45.594320,-122.678556,7644510.0,710295.0,2024-01-01,1
56671,139602,1800 BLOCK OF SW SKYLINE BLVD,24-900059,Property,Sylvan-Highlands,2024-01-01,2328,Larceny Offenses,Shoplifting,45.509787,-122.735427,7629105.0,679883.0,2024-01-01,1
56672,136472,1400 BLOCK OF NW HOYT ST,24-634,Property,Pearl,2024-01-01,1926,Motor Vehicle Theft,Motor Vehicle Theft,45.527127,-122.685830,7641986.0,685855.0,2024-01-01,1
56673,152254,4500 BLOCK OF NE FREMONT ST,24-530,Property,Beaumont-Wilshire,2024-01-01,1603,Larceny Offenses,Theft of Motor Vehicle Parts or Accessories,45.548334,-122.616426,7659969.0,693111.0,2024-01-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
587388,587388,SW MAIN ST / SW VISTA AVE,23-336292,Property,Goose Hollow,2023-12-30,1030,Motor Vehicle Theft,Motor Vehicle Theft,45.520588,-122.698208,7638750.0,683558.0,2023-12-31,1
587636,587636,UNKNOWN ADDRESS,23-330843,Person,Other,2023-02-01,0,Assault Offenses,Intimidation,NaN,NaN,NaN,NaN,2023-12-31,1
587643,587643,UNKNOWN ADDRESS,23-681030,Property,Other,2023-12-31,0,Fraud Offenses,Credit Card/ATM Fraud,NaN,NaN,NaN,NaN,2023-12-31,1
587821,587821,UNKNOWN ADDRESS,23-931937,Property,Other,2023-12-31,2015,Larceny Offenses,Theft From Motor Vehicle,NaN,NaN,NaN,NaN,2023-12-31,1
